In [ ]:
# default_exp database

In [ ]:
#hide
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
#%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Database for tweets storage

> Use PostgreSQL to store tweets

In [ ]:
#export
from os import getenv, environ
from dotenv import load_dotenv
import psycopg2
import logging
import pandas as pd

In [ ]:
#export
# Logging
logger = logging.getLogger("tweet-archiveur")
logFormatter = logging.Formatter("%(asctime)s -  %(name)-12s %(levelname)-8s %(message)s")
logger.setLevel(logging.DEBUG)
# # File logger
# fh = logging.FileHandler("tweet-archiveur.log")
# fh.setLevel(logging.DEBUG)
# fh.setFormatter(logFormatter)
# logger.addHandler(fh)
if not len(logger.handlers):
    # Console logger
    consoleHandler = logging.StreamHandler()
    consoleHandler.setFormatter(logFormatter)
    logger.addHandler(consoleHandler)
logger.info(f'Loading database module...')

2021-03-20 12:10:20,375 -  tweet-archiveur INFO     Loading database module...


In [ ]:
#hide
# Load .env only in Notebook, it will be populated at runtime by docker
from pathlib import Path
env_path = Path('..') / '.env'
if env_path.is_file():
    load_dotenv(dotenv_path=env_path)
else:
    logger.error(f"No {env_path} found !")

# Force some variable outside Docker
environ["DATABASE_PORT"] = '8479'
environ["DATABASE_HOST"] = 'localhost'
environ["DATABASE_USER"] = 'tweet_archiveur_user'


True

In [ ]:
#export
ENV = [
    "DATABASE_USER",
    "DATABASE_PASS",
    "DATABASE_HOST",
    "DATABASE_PORT",
    "DATABASE_NAME",
    "DATABASE_URL",
]

# Global variables that will be populated when database_url is called by db_connect
db_schema = None
twitter_users_table = None
tweets_table = None
hashtags_table = None


def database_config():
    return tuple(getenv(env) for env in ENV)


def database_url() -> str:
    global db_schema, tweets_table, twitter_users_table, hashtags_table
    user, pswd, host, port, name, url = database_config()
    db_schema = getenv('DATABASE_USER')
    twitter_users_table = 'twitter_users'
    tweets_table = 'tweets'
    hashtags_table = 'tweets_hashtags'
    
    logger.debug(f"DEBUG : connect(user={user}, password=XXXX, host={host}, port={port}, database={name}, url={url})")
    if user is None and url is None:
        logger.error("Empty .env : no user or URL !")
        return None
    if url:
        return url
    else:
        return f"postgresql://{user}:{pswd}@{host}:{port}/{name}"

def db_connect():
    return psycopg2.connect(database_url())
    

In [ ]:
#export

def exec_query(conn, sql):
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()
    cur.close()

# https://stackoverflow.com/questions/1874113/checking-if-a-postgresql-table-exists-under-python-and-probably-psycopg2
# def is_table_exist(conn, table_str):
#     exists = False
#     try:
#         cur = conn.cursor()
#         cur.execute(f"select * from information_schema.tables where table_schema='{db_schema}' and  table_name='{table_str}';")
#         if cur.fetchone() is None and cur:
#             return False
#         exists = cur.fetchone()[0]
#         cur.close()
#     except psycopg2.Error as e:
#         logger.error(e)
#         return False
#     return exists


def create_tables_if_not_exist(conn, force = False):
    exec_query(conn, f'CREATE SCHEMA IF NOT EXISTS {db_schema}  AUTHORIZATION CURRENT_USER;')
    if force :
        logger.info("Cleaning database")
        # Drop table if exist
        exec_query(conn, f'DROP TABLE IF EXISTS {db_schema}.{hashtags_table};')
        exec_query(conn, f'DROP TABLE IF EXISTS {db_schema}.{tweets_table};')
        exec_query(conn, f'DROP TABLE IF EXISTS {db_schema}.{twitter_users_table};')
        
        

    # Create table
    #if not is_table_exist(conn, 'twitter_users'):
    users = f'''
    CREATE TABLE IF NOT EXISTS {db_schema}.{twitter_users_table}
    (
        twitter_id bigint NOT NULL,
        name character varying(50) NOT NULL,
        twitter_followers integer,
        twitter_tweets integer,
        PRIMARY KEY (twitter_id)
    );
    '''
    exec_query(conn, users)
    exec_query(conn, f'ALTER TABLE {db_schema}.{twitter_users_table} OWNER to CURRENT_USER;')
    #if not is_table_exist(conn, tweets_table):
    tweets = f'''
    CREATE TABLE IF NOT EXISTS {db_schema}.{tweets_table}
    (
        tweet_id bigint NOT NULL,
        twitter_id bigint NOT NULL,
        datetime_utc timestamp without time zone,
        datetime_local timestamp without time zone,
        retweet integer,
        favorite integer,
        text character varying(500),
        PRIMARY KEY (tweet_id),
        CONSTRAINT fk_user
         FOREIGN KEY(twitter_id)
             REFERENCES {twitter_users_table}(twitter_id)
    );'''
    exec_query(conn, tweets)
    exec_query(conn, f'ALTER TABLE {db_schema}.{tweets_table} OWNER to CURRENT_USER;')
    #if not is_table_exist(conn, hashtags_table):
    hashtags = f'''
    CREATE TABLE IF NOT EXISTS {db_schema}.{hashtags_table}
    (
        id SERIAL PRIMARY KEY,
        tweet_id bigint NOT NULL,
        twitter_id bigint NOT NULL,
        datetime_local timestamp without time zone,
        hashtag character varying(140),
        CONSTRAINT fk_user
         FOREIGN KEY(twitter_id)
             REFERENCES {twitter_users_table}(twitter_id),
        CONSTRAINT fk_tweet
         FOREIGN KEY(tweet_id)
             REFERENCES {tweets_table}(tweet_id)
    );'''
    exec_query(conn, hashtags)
    exec_query(conn, f'ALTER TABLE {db_schema}.{hashtags_table} OWNER TO CURRENT_USER;')

In [ ]:
#hide
conn = db_connect()
create_tables_if_not_exist(conn, force=False)
conn.close()

2021-03-20 12:10:20,437 -  tweet-archiveur DEBUG    DEBUG : connect(user=tweet_archiveur_user, password=XXXX, host=localhost, port=8479, database=tweet_archiveur, url=None)


In [ ]:
#export

# Bulk INSERT of values in a table
def insert_pandas(conn, table, df, fields, prevent_conflict = True, delete_where = None):
    """
    Using cursor.mogrify() to build the bulk insert query
    then cursor.execute() to execute the query
    Thanks to https://naysan.ca/2020/05/09/pandas-to-postgresql-using-psycopg2-bulk-insert-performance-benchmark/
    """
    cursor = conn.cursor()
    if delete_where is not None:
        query  = f'DELETE FROM {table} WHERE {list(delete_where.values())[0]} IN ({", ".join([str(i) for i in df.head(2).tweet_id.tolist()])})'
        cursor.execute(query)
    # Create a list of tupples from the dataframe values
    col = "'" + "', '".join(fields.keys()) + "'"
    df = eval("df[[" + col + "]]")
    #logger.debug(f"Bulk insert of {len(df)} lines of {len(df.columns)} columns.")
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(fields.values())
    # SQL quert to execute

    param_type = param = "(" + ",".join(['%s' for i in range(len(df.columns))]) + ")" 
    values = [cursor.mogrify(param_type, tup).decode('utf8') for tup in tuples]
    query  = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)
    # Get the primary key, we suppose it is the first one
    primary_key = list(fields.values())[0]
    # Get the list of other column, excluding the primary
    other_fields = list(fields.values())[1:]
    # Build the query to UPDATE if the line already exist
    if prevent_conflict:
        query += f' ON CONFLICT ({primary_key}) DO UPDATE SET '
        query += "(" + ", ".join(other_fields) + ")"
        excluded = ['EXCLUDED.' + col for col in other_fields]
        query += ' = (' + ", ".join(excluded) + ")"
    query += ';'
    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

In [ ]:
#export
import string
printable = set(string.printable)
printable.remove('%')
# https://www.programiz.com/python-programming/methods/built-in/filter"
def filter_str(s):
    #return "".join(filter(lambda x: x in printable, s))
    s = s.replace('%', '%%')
    return s

In [ ]:
#export
def insert_twitter_users(conn, df):
    if len(df) < 1:
        logger.debug('insert_twitter_users : Empty dataframe')
        return
    table = f'{db_schema}.{twitter_users_table}'
    fields = { # pandas : database
        'twitter_id' : 'twitter_id',
        'nom' : 'name',
        'twitter_followers' : 'twitter_followers',
        'twitter_tweets' : 'twitter_tweets' 
    }
    insert_pandas(conn, table, df, fields)


def insert_tweets(conn, df):
    if len(df) < 1:
        logger.debug('insert_tweets : Empty dataframe')
        return
    table = f'{db_schema}.{tweets_table}'
    df['text_new'] = df.text.apply(filter_str)
    fields = { # pandas : database
        'tweet_id' : 'tweet_id',
        'twitter_id' : 'twitter_id',
        'datetime_utc' : 'datetime_utc',
        'datetime_local' : 'datetime_local',
        'retweet' : 'retweet',
        'favorite' : 'favorite',
        'text_new' : 'text',
    }
    insert_pandas(conn, table, df, fields)
    
def insert_hashtags(conn, df):
    if len(df) < 1:
        logger.debug('insert_hashtags : Empty dataframe')
        return
    table = f'{db_schema}.{hashtags_table}'
    fields = { # pandas : database
        'tweet_id' : 'tweet_id',
        'twitter_id' : 'twitter_id',
        'datetime_local' : 'datetime_local',
        'hashtag' : 'hashtag',
    }
    delete_where = { # pandas : database
        'tweet_id' : 'tweet_id',
    }
    insert_pandas(conn, table, df, fields, prevent_conflict = False, delete_where = delete_where)

In [ ]:
#hide
conn = db_connect()
# Load users
df = pd.read_csv('https://github.com/regardscitoyens/twitter-parlementaires/raw/master/data/deputes.csv')#.head(3)
insert_twitter_users(conn, df)

# Load Tweets
df = pd.read_csv('tweets-sample.csv')#.head(2)
insert_tweets(conn, df)

# Load Hashtags
df = pd.read_csv('hashtags-sample.csv')
insert_hashtags(conn, df)

conn.close()

2021-03-20 12:10:20,515 -  tweet-archiveur DEBUG    DEBUG : connect(user=tweet_archiveur_user, password=XXXX, host=localhost, port=8479, database=tweet_archiveur, url=None)
2021-03-20 12:10:20,973 -  tweet-archiveur DEBUG    Bulk insert of 603 lines of 4 columns.
2021-03-20 12:10:20,999 -  tweet-archiveur DEBUG    Bulk insert of 100 lines of 7 columns.
2021-03-20 12:10:21,009 -  tweet-archiveur DEBUG    Bulk insert of 121 lines of 4 columns.


In [ ]:
#hide
", ".join([str(i) for i in df.head(2).tweet_id.tolist()])

'1372621356223885322, 1372621356223885322'

In [ ]:
#hide
#######################################################
# DEBUG CODE BELOW

In [ ]:
#hide
fields = { # pandas : database
    'tweet_id' : 'tweet_id',
    'twitter_id' : 'twitter_id',
    'datetime_local' : 'datetime_local',
    'hashtag' : 'hashtag',
}
col = "'" + "', '".join(fields.keys()) + "'"
print(col)
df2 = eval("df[[" + col + "]]")
tuples = [tuple(x) for x in df2.to_numpy()]
tuples[0]

'tweet_id', 'twitter_id', 'datetime_local', 'hashtag'


(1372621356223885322, 76584619, '2021-03-18 19:50:08', 'Castex19h')

In [ ]:
# hide
query = "test "
primary_key = list(fields.values())[0]
other_fields = list(fields.values())[1:]
query += f' ON CONFLICT ({primary_key}) DO UPDATE SET '
query += "(" + ", ".join(other_fields) + ")"
excluded = ['EXCLUDED.' + col for col in other_fields]
query += ' = (' + ", ".join(excluded) + ");"
# (col2, col3, col4) = (EXCLUDED.col2, EXCLUDED.col3, EXCLUDED.col4);
query

'test  ON CONFLICT (tweet_id) DO UPDATE SET (twitter_id, datetime_local, hashtag) = (EXCLUDED.twitter_id, EXCLUDED.datetime_local, EXCLUDED.hashtag);'

In [ ]:
#hide
fields.values()

dict_values(['tweet_id', 'twitter_id', 'datetime_local', 'hashtag'])

In [ ]:
#hide
list(fields.values())[1:]

['twitter_id', 'datetime_local', 'hashtag']

In [ ]:
#hide
array = [1372621356223885322, 272621356223885322, 372621356223885322]
_ = array.pop(0)
array.reverse()

import random
random.shuffle(array)
array

[372621356223885322, 272621356223885322]